# Imports

In [49]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, solo_encabezado, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from nlppen.extraccion.utils.extraerFechaRecibido import ExtraerFecha
from pyspark.sql.functions import length
from nlppen.extraccion.utils.misc import splitResolucion

import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configuracion Spark

In [50]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "6")
         .config("spark.executor.memory", "6g")
         .config("spark.executor.cores", "12")
         .config("spark.driver.memory", "12g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "16g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [51]:
ejecutar = False
if ejecutar:
    terminos = {
        'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
        'se condena': [r'\bse condena\b', r'\bse le condena\b', r'\bse les condena\b'],
        'plan': [r'\bplan\b'],
        'plazo': [r'\bplazo\b', r'\bt[éermino (improrrogable)? (de)?\b']
    }
    seleccion = Seleccion(terminos, spark, parquet_path='./dataset/partition', datasets_path='./datasets/estructurales')
    print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
    seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True)
    print("Cantidad elementos despues de filtrados : " + str(seleccion.sdf.count()))
    estructurales = SentenciasEstructurales(seleccion)

# Procesamiento de todas las variables de sentencias estructurales

## La siguiente celda ejecuta el cálculo de todas las variables definidas para las sentencias estructurales

Nota: Modificar la bandera "ejecutar" para aplicar todo el procesamiento.

In [52]:
from pyspark.sql.types import *

if ejecutar:
    columnas = {
        'se ordena PER' : ArrayType(StringType()),
        'se ordena LOC' : ArrayType(StringType()),
        'se ordena ORG' : ArrayType(StringType()),
        'se ordena MISC' : ArrayType(StringType()),
        'se ordena GPE' : ArrayType(StringType()),
        'se ordena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeOrdena(columnas, True, True)

    columnas = {
        'se condena PER' : ArrayType(StringType()),
        'se condena LOC' : ArrayType(StringType()),
        'se condena ORG' : ArrayType(StringType()),
        'se condena MISC' : ArrayType(StringType()),
        'se condena GPE' : ArrayType(StringType()),
        'se condena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeCondena(columnas, True, True)

    columnas = {
        'extension sentencia' : IntegerType(),
        'extension por lo tanto' : IntegerType()
    }
    estructurales.extraerExtension(columnas, True)

    columnas = {
        'plazosDefinidos' : ArrayType(TimestampType()) #Correción día 27-11-2021. Correr de nuevo
    }
    estructurales.plazosDefinidos(columnas, True)

    columnas = {
        'FechaSolicitud' : TimestampType(),
    }

    estructurales.extrarFechaRecibido(columnas, True)

    columnas = {
        'num resolucion' : StringType()
    }

    estructurales.extraerNumeroSentencia(columnas, True)

    columnas = {
        'inst internacionales' : ArrayType(StringType())
    }

    estructurales.extraerInstrumentosInternacionales(columnas, True)

    columnas = {
        'derechos Norm' : ArrayType(StringType()),
        'derechos GenXPat' : ArrayType(StringType())
    }
    estructurales.extraerDerechos(columnas, True)

    columnas = {
        'derechos Acotados' : ArrayType(StringType()),
        'derechos General' : ArrayType(StringType()),
        'derechos Fundamental' : ArrayType(StringType()),
        'derechos Humano' : ArrayType(StringType())
    }

    estructurales.extraerDerechosSinNormalizar(columnas, True)
    
    sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
    columnas = {
        'id_sentencia' : StringType(),
        'num_doc_oficial' : StringType(),
        'expediente_oficial' : StringType()
    }
    estructurales.agregarIDSentencia(columnas, sentencias, True)
    
    columnas = {
        'citasVotDate' : MapType(StringType(), TimestampType()), #Dicc de Numero de voto y Fecha
        'citasIDVoto' : MapType(StringType(), StringType()) #Dicc de id_Sentencia y # de voto
    }
    estructurales.extrarCitaSentenciasFecha(columnas, sentencias, True)
    
    columnas = {
        'seguimientoExt' : StringType(), #Texto extraido de seguimiento
        'seguimientoValue' : IntegerType() #Cantidad de ocurrencias
    }
    estructurales.extraerSeguimiento(columnas, True)
    #estructurales.seleccion.sdf.toPandas().head(5)
    #Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
    #estructurales.seleccion.guardarDatos()

In [64]:
#writer = estructurales.seleccion.sdf.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')

In [54]:
#df = spark.read.parquet("./dataset/procesado")
#df.toPandas().head()

## Guardar los resultados del preprocesamiento por año

In [55]:
#if ejecutar:
#    df = spark.read.parquet(seleccion.datasets_path+"/terminos.parquet")
#    writer = df.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')


## Cargar los resultados preprocesados y divididos por año

In [79]:
if not ejecutar:
    seleccion = Seleccion({}, spark, parquet_path='./dataset/procesado', datasets_path='./datasets/estructuralesx')
    estructurales = SentenciasEstructurales(seleccion)

In [80]:
estructurales.seleccion.sdf.select("id_sentencia", "expediente_oficial").toPandas().head(5)

,id_sentencia,expediente_oficial
0,sen-1-0007-893680,180169790007CO
1,sen-1-0007-860153,180157550007CO
2,sen-1-0007-894916,180178880007CO
3,sen-1-0007-896941,180188180007CO
4,sen-1-0007-853795,180063820007CO


## Ejecutar un criterio

In [82]:
from pyspark.sql.types import *
if not ejecutar:
    columnas = {
        'plazosDefinidos' : ArrayType(TimestampType()) #Correción día 27-11-2021. Correr de nuevo
    }
    estructurales.plazosDefinidos(columnas, True, True)
    s = estructurales.seleccion.sdf
    s.toPandas().head(5)

In [83]:
s = estructurales.seleccion.sdf
print(s.filter(s.plazo > 0).count())
print(s.filter(s.plazosDefinidos.isNotNull()).count())
s.filter(s.plazosDefinidos.isNotNull()).filter(s.plazo == 0).select( "id_sentencia", "plazosDefinidos" , "plazo").toPandas()

709
631


,id_sentencia,plazosDefinidos,plazo


# Cargar procesamiento de sentencias estructurales.

# Zona de trabajo y cosas utiles

### Sobreescribir el dataset

In [69]:
#writer = estructurales.seleccion.sdf.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')

### Sobreescribir terminos

In [71]:
"""terminos = {
        'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
        'se condena': [r'\bse condena\b', r'\bse le condena\b', r'\bse les condena\b'],
        'plan': [r'\bplan\b'],
        'plazo': [r'\bplazo\b', r'\bt[éermino (improrrogable)? (de)?\b']
    }
    seleccion = Seleccion(terminos, spark, parquet_path='./dataset/procesado', datasets_path='./datasets/estructuralesx')
    #seleccion = Seleccion(terminos, spark, parquet_path='./dataset/partition', datasets_path='./datasets/estructurales')
    print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
    seleccion.filtrar_sentencias(preprocess=solo_portanto, precargar=False, keepRowEmpty=True, overwriteColumns=True)
    estructurales.seleccion = seleccion"""

'terminos = {\n        \'se ordena\': [r\'\x08se ordena\x08\', r\'\x08se le ordena\x08\', r\'\x08se les ordena\x08\'],\n        \'se condena\': [r\'\x08se condena\x08\', r\'\x08se le condena\x08\', r\'\x08se les condena\x08\'],\n        \'plan\': [r\'\x08plan\x08\'],\n        \'plazo\': [r\'\x08plazo\x08\', r\'\x08t[éermino (improrrogable)? (de)?\x08\']\n    }\n    seleccion = Seleccion(terminos, spark, parquet_path=\'./dataset/procesado\', datasets_path=\'./datasets/estructuralesx\')\n    #seleccion = Seleccion(terminos, spark, parquet_path=\'./dataset/partition\', datasets_path=\'./datasets/estructurales\')\n    print("Cantidad elementos originales : " + str(seleccion.sdf.count()))\n    seleccion.filtrar_sentencias(preprocess=solo_portanto, precargar=False, keepRowEmpty=True, overwriteColumns=True)\n    estructurales.seleccion = seleccion'

### Funciones pandas y visualizacion de datos

In [ ]:
"""from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)
"""


### Funcion de testing

In [25]:
from spacy.matcher import Matcher
from spacy.tokens import Span
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.extraccion.utils.Txt2Date import Txt2Date
import re
def testing(res, txt, patterns):
    regularExpresion = re.compile(r"\.|h[óo]r[aá]s?|d[íi][áa]s|m[ée]s([ée]s)?|[áa]ñ[óo]s?")

    txt = txt.lower()
    plazos = []
    #No procesar las que son sin lugar.
    if res in ["Con lugar", "Con lugar parcial"]:
        doc = nlp(txt)
        matcher = Matcher(nlp.vocab)
        matcher.add("Patron 1 :", patterns, greedy="FIRST")

        matches = matcher(doc)

        #Crear los objetos para convertir de texto a número y de para convertir de texto a fechas.
        convertToDate = Txt2Date()
        convertToNumber = Txt2Numbers()
        for _, start, end in matches:
            print("Match: ", doc[start:end].text)
            includeText = False
            plazo = ""
            stringNumber = ""
            stringNumberFormat = None
            for token in doc[start:end]:
                # Recolectar todos los token a partir del primer NUM hasta considir con
                # algunas de las palabras de: [horas, dias, meses, año]
                # eg( tres dias, cuatro meses)
                if includeText:
                    if token.pos_ == "PUNCT":
                        break
                    if regularExpresion.search(token.text) != None:
                        if stringNumberFormat is None:
                            number = convertToNumber.number(textToken)
                        else:
                            number = stringNumberFormat
                        if number != None: 
                            deltaTime = convertToDate.txt2Date(token.text, number)
                            # Convierte los delta time a un datatime y se obtiene el timestamp
                            plazo = pd.Timestamp(pd.to_datetime('1970-01-01') + deltaTime).to_pydatetime()
                        
                        break
                    stringNumber += " " + token.text
                else:
                    if token.pos_ == "NUM":
                        textToken = token.text 
                        print(textToken)
                        if textToken.isdigit() == False:
                            stringNumber += textToken
                            includeText = True
                        else:
                            stringNumberFormat = int(textToken)
                            includeText = True
                    else:
                        if token.text == "un":
                            try:
                                nextWord = doc[token.i + 1].text
                                print(nextWord)
                                if regularExpresion.search(nextWord) != None:
                                    print("si")
                                    stringNumberFormat = 1
                                    includeText = True
                            except:
                                pass
            if plazo != "":
                plazos.append(plazo)
        print(plazos)

### Recorrer dataset con una funcion de testing

In [33]:
citas = s.filter(s.plazosDefinidos.isNull()).filter(s.plazo > 0).filter(s.plazosDefinidos2.isNull()).toPandas()
#citas = s.filter(s.CitaSentencias_ents.isNotNull()).toPandas()
nlp = spark_get_spacy("es_core_news_lg")
#size = len(citas)
size = 1
for i in range(0, 1):
    sent = citas.iloc[i, 4]
    txt = citas.iloc[i, 1]
    print("******************************")
    print(sent)
    termino_ext = citas.iloc[i, 8]
    plazos_pattern =  [{"LOWER": "dentro", "OP":"?"}, 
                       {"IS_STOP": True, "OP":"*"}, 
                       {"LOWER": {"IN":["plazo", "término", "termino"]}},
          {"TEXT": {"REGEX": "^(?!,|\.|h[óo]r[aá]s?|d[íi][áa]s|m[ée]s([ée]s)?|[áa]ñ[óo]s?)"}, "OP": "+"},
          #{"TEXT": {"REGEX": "^(?!\.|[Hh][OoÓó][Rr][Aa]([Ss])?|[dD][iíIÍ][ÁAaa]([Ss])?|[Mm][EeÉé][Ss]([EeÉé][Ss])?|[ÁáAa][Ññ][OoÓó]([Ss])?)"}, "OP": "+"},
          #{"TEXT": {"REGEX": "\.|[Hh][OoÓó][Rr][Aa]([Ss])?|[dD][iíIÍ][ÁAaa]([Ss])?|[Mm][EeÉé][Ss]([EeÉé][Ss])?|[ÁáAa][Ññ][OoÓó]([Ss])?"}},
          {"TEXT": {"REGEX": ",|\.|h[óo]r[aá]s?|d[íi][áa]s|m[ée]s([ée]s)?|[áa]ñ[óo]s?"}, "OP": "+"}
          ]
    patterns  = [plazos_pattern]
    txt += " además en el término de DOS AÑOS reporte la vara" 
    testing(termino_ext, solo_portanto(txt), patterns)

******************************
180087690007CO
Match:  por el plazo señalado por su médico tratante,
Match:  además en el término de dos años
dos
[datetime.datetime(1972, 1, 1, 0, 0)]


### Cargar spacy

In [68]:
"""nlp = spark_get_spacy('es_core_news_lg')
doc = nlp("plan remedial")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
"""

'nlp = spark_get_spacy(\'es_core_news_lg\')\ndoc = nlp("plan remedial")\nfor token in doc:\n    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)\n'